In [5]:
import pandas as pd
df = pd.read_csv('accidents.csv')
df

FileNotFoundError: [Errno 2] No such file or directory: 'accidents.csv'

In [11]:
df_raw = df.groupby(by=['State','Year']).sum()[['Other_CC','Total_CC','FDH']].reset_index()

In [12]:
df_single_2020 = df_raw[df_raw['Year']==2020]

In [13]:
df_all_years = pd.DataFrame()
for i in range(2000,2021):
    df_single_2020['Year'] = i
    df_all_years = pd.concat([df_all_years,df_single_2020])
    
df_all_years = df_all_years.reindex()

C:\Users\ese95\AppData\Local\Temp\ipykernel_32740\2177124246.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_single_2020['Year'] = i


In [14]:
df_all_years

,State,Year,Other_CC,Total_CC,FDH
1,AK,2000,41,60,41
3,AL,2000,431,20185,431
5,AR,2000,412,3090,412
7,AZ,2000,130,29535,130
9,CA,2000,494,285532,494
...,...,...,...,...,...
91,VT,2020,80,186,80
93,WA,2020,271,13211,271
95,WI,2020,427,3933,427
97,WV,2020,212,1171,212


In [3]:
import pandas as pd
ccdf = pd.read_csv('cc_cty_panel_.csv')

In [4]:
ccdf.columns

Index(['state', 'cty_fips', 'year', 'pop04', 'center_count_cc',
       'capacity_total_cc', 'center_count_fdh', 'capacity_total_fdh',
       'center_count_other', 'capacity_total_other', 'center_tot',
       'capacity_total', 'cc_desert'],
      dtype='object')

In [43]:
poprepopulate = ccdf.groupby(by=['state','year']).sum()[['pop04','center_count_cc','center_count_fdh','center_count_other']]
idxs = poprepopulate.index

mapfipcode = pd.read_csv('state_fips.csv')
inusefips = set(mapfipcode['id'])
in_data_state = set(poprepopulate.index.get_level_values(level='state'))
fips_lack = inusefips - in_data_state

for state in inusefips:
    prev_pop = 0    
    prev_total = 0
    prev_other = 0
    prev_fdh = 0
    for yr in range(2000,2024):
        pair = (state,yr)
        if pair not in idxs:
            poprepopulate.loc[(state,yr),'pop04'] = prev_pop
            poprepopulate.loc[(state,yr),'center_count_cc'] = prev_total
            poprepopulate.loc[(state,yr),'center_count_fdh'] = prev_other
            poprepopulate.loc[(state,yr),'center_count_other'] = prev_fdh
        else:
            prev_pop = poprepopulate.loc[(state,yr),'pop04']
            prev_total = poprepopulate.loc[(state,yr),'center_count_cc']
            prev_other = poprepopulate.loc[(state,yr),'center_count_fdh'] 
            prev_fdh = poprepopulate.loc[(state,yr),'center_count_other']

In [44]:
poprepopulate.reset_index().to_csv('final_trend2.csv')

In [40]:
popwithCode = pd.merge(mapfipcode, poprepopulate, left_on='id', right_on='state')[['name', 'code', 'state','year', 'pop04']]
pd.merge(popwithCode,df_all_years,left_on=['code','year'],right_on=['State','Year']).columns



Index(['name', 'code', 'state', 'year', 'pop04', 'State', 'Year', 'Other_CC',
       'Total_CC', 'FDH'],
      dtype='object')

In [42]:
pd.merge(popwithCode,df_all_years,left_on=['code','year'],right_on=['State','Year'])[['name', 'code', 'state', 'year', 'pop04', 'Other_CC','Total_CC', 'FDH']].to_csv('ChildcareCaseData.csv')

In [13]:
import pandas as pd

stateLevel = pd.read_csv('stateCaseData.csv')
alsoStateLevel = pd.read_csv('ChildcareCaseData.csv') # not using this file, but will need for population scale up line
countyLevel = pd.read_csv('countyCaseData.csv')




In [4]:
stateLevel.head()

,State,Year,Total_CC,ln_name,fipscode,Other_CC,FDC
0,AL,2019,3,Alabama,1,6,6
1,AL,2020,9,Alabama,1,3,9
2,AK,2019,6,ALASKA,2,4,6
3,AK,2020,6,ALASKA,2,6,9
4,AR,2019,9,Arkansas,5,7,6


In [7]:
countyLevel.head()

,fip,County,State,Year,Other_CC,Total_CC,FDH
0,3473,Baldwin,AL,2019,4,7,4
1,3493,Chilton,AL,2019,4,11,4
2,3558,Lauderdale,AL,2019,3,2,3
3,3625,Butler,AL,2019,3,71,3
4,3629,Tuscaloosa,AL,2019,3,250,3


In [21]:
presetgroup = countyLevel.groupby(by=['State','Year']).sum()[['Other_CC','Total_CC','FDH']].reset_index()
presetgroup.head()

,State,Year,Other_CC,Total_CC,FDH
0,AK,2019,60,58,60
1,AK,2020,41,60,41
2,AL,2019,437,19238,437
3,AL,2020,431,20185,431
4,AR,2019,407,991,407


In [28]:
presetgroup.merge(stateLevel[['State','Year','ln_name','fipscode']],on=['State','Year']).to_csv('statelevel2.csv')

In [27]:
stateLevel.columns

Index(['State', 'Year', 'Total_CC', 'ln_name', 'fipscode', 'Other_CC', 'FDC'], dtype='object')

In [6]:
import pandas as pd
df = pd.read_csv('cc_cty_panel_.csv')

In [8]:
trend = df.groupby(by=['state','year']).sum()['pop04']

In [11]:
trend = trend.reset_index()

In [13]:
df.head().columns

Index(['state', 'cty_fips', 'year', 'pop04', 'center_count_cc',
       'capacity_total_cc', 'center_count_fdh', 'capacity_total_fdh',
       'center_count_other', 'capacity_total_other', 'center_tot',
       'capacity_total', 'cc_desert'],
      dtype='object')

In [23]:
cases = df.groupby(by=['state','year']).sum()[['capacity_total_cc','capacity_total_fdh','capacity_total_other']].fillna(0).reset_index()


In [31]:
trend_data = trend.merge(cases,on=['state','year'])
trend_data.head().columns

Index(['state', 'year', 'pop04', 'capacity_total_cc', 'capacity_total_fdh',
       'capacity_total_other'],
      dtype='object')

In [33]:
# map state name and fips
stateFips = pd.read_csv('state_fips.csv')
final_trend_dataset = pd.merge(left=trend_data,right=stateFips,left_on='state',right_on='id')[['state', 'year', 'pop04', 'capacity_total_cc', 'capacity_total_fdh', 'capacity_total_other','code']]

In [34]:
final_trend_dataset.to_csv('final_trend.csv')

In [4]:
import pandas as pd
pd.read_csv('final_trend.csv')

,fips,Year,pop2,Total_CC,FDH,Other_CC,State,pop
0,1,2000,3519,54,0,0,AL,3
1,1,2001,3471,54,0,0,AL,3
2,1,2002,3537,54,0,0,AL,3
3,1,2003,46628,54,0,0,AL,46
4,1,2004,46477,54,0,0,AL,46
...,...,...,...,...,...,...,...,...
772,55,2019,333184,144373,2635,0,WI,333
773,55,2020,331066,142533,2167,0,WI,331
774,55,2021,329324,143400,2132,0,WI,329
775,55,2022,321594,143100,2058,0,WI,321


In [ ]:
final_trend_dataset[final_trend_dataset['']]